<a href="https://colab.research.google.com/github/ianellisjones/usn/blob/main/Fleet_Tracker_Map_(Improved).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
"""
US NAVY UNIFIED FLEET INTEL (Official + OSINT)

A comprehensive intelligence collector that runs two parallel operations:
1. OFFICIAL: Scrapes uscarriers.net for verified deployment history (CVN/LHA/LHD).
2. OSINT: Scrapes WarshipCam and Search Engines for real-time social sightings.

Outputs two CSV files: 'big_deck_status.csv' and 'social_naval_intel.csv'.
"""

import csv
import re
import time
import random
from pathlib import Path
from typing import List, Tuple, Dict, Optional

import requests
from bs4 import BeautifulSoup

# ==============================================================================
# CONFIGURATION
# ==============================================================================

USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'

# --- OFFICIAL SOURCES (USCarriers.net) ---
FLEET_URLS: List[str] = [
    # Aircraft Carriers (CVN)
    "http://uscarriers.net/cvn68history.htm", # USS Nimitz
    "http://uscarriers.net/cvn69history.htm", # USS Eisenhower
    "http://uscarriers.net/cvn70history.htm", # USS Carl Vinson
    "http://uscarriers.net/cvn71history.htm", # USS Theodore Roosevelt
    "http://uscarriers.net/cvn72history.htm", # USS Abraham Lincoln
    "http://uscarriers.net/cvn73history.htm", # USS George Washington
    "http://uscarriers.net/cvn74history.htm", # USS John C. Stennis
    "http://uscarriers.net/cvn75history.htm", # USS Harry S. Truman
    "http://uscarriers.net/cvn76history.htm", # USS Ronald Reagan
    "http://uscarriers.net/cvn77history.htm", # USS George H.W. Bush
    "http://uscarriers.net/cvn78history.htm", # USS Gerald R. Ford

    # Amphibious Assault Ships (LHA/LHD)
    "http://uscarriers.net/lhd1history.htm", # USS Wasp
    "http://uscarriers.net/lhd2history.htm", # USS Essex
    "http://uscarriers.net/lhd3history.htm", # USS Kearsarge
    "http://uscarriers.net/lhd4history.htm", # USS Boxer
    "http://uscarriers.net/lhd5history.htm", # USS Bataan
    "http://uscarriers.net/lhd7history.htm", # USS Iwo Jima
    "http://uscarriers.net/lhd8history.htm", # USS Makin Island
    "http://uscarriers.net/lha6history.htm", # USS America
    "http://uscarriers.net/lha7history.htm", # USS Tripoli
]

# --- OSINT SOURCES ---
WARSHIPCAM_URL = "https://www.warshipcam.net/"

# ==============================================================================
# MODULE 1: OFFICIAL TRACKER LOGIC
# ==============================================================================

def fetch_history_text(url: str, char_limit: int = 50000) -> str:
    try:
        response = requests.get(url, headers={'User-Agent': USER_AGENT}, timeout=20)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        full_text = soup.get_text(separator='\n')
        lines = [line.strip() for line in full_text.split('\n') if line.strip()]
        clean_text = '\n'.join(lines)
        return clean_text[-char_limit:] if len(clean_text) > char_limit else clean_text
    except Exception as e:
        return f"ERROR: {str(e)}"

def parse_status_entry(text_block: str) -> Tuple[str, str]:
    lines = text_block.split('\n')
    current_year = "Unknown"
    years_found = re.findall(r'(202[3-6])', text_block)
    if years_found:
        priority_years = [y for y in years_found if y in ['2024', '2025']]
        current_year = priority_years[-1] if priority_years else years_found[-1]

    processed_lines = []
    running_year = current_year
    for line in lines:
        year_match = re.search(r'^202[3-6]', line)
        if year_match: running_year = year_match.group(0)
        processed_lines.append({'text': line, 'year': running_year})

    keywords = ["moored", "anchored", "underway", "arrived", "departed", "transited", "operations", "returned", "participated", "conducted", "moved to", "visited", "pulled into", "sea trials", "flight deck certification"]
    allowed_years = ["2024", "2025", "2026"]

    for entry in reversed(processed_lines):
        text_lower = entry['text'].lower()
        year = entry['year']
        if year in allowed_years and any(k in text_lower for k in keywords):
            if text_lower.strip().startswith("from ") and " - " in text_lower: continue
            return year, entry['text']
    return current_year, "No status found."

def categorize_location(text: str) -> str:
    text = text.lower()
    if "departed san diego" in text: return "Pacific Ocean"
    if "departed norfolk" in text: return "Atlantic Ocean"
    if "departed pearl harbor" in text: return "Pacific Ocean"
    if "departed mayport" in text: return "Atlantic Ocean"

    location_map = {
        "Norfolk / Portsmouth": ["norfolk", "portsmouth", "virginia beach", "nassco"],
        "San Diego": ["san diego", "north island", "camp pendleton"],
        "Bremerton / Kitsap": ["bremerton", "kitsap"],
        "Newport News": ["newport news"],
        "Yokosuka": ["yokosuka"],
        "Pearl Harbor": ["pearl harbor"],
        "Mayport": ["mayport"],
        "Everett": ["everett"],
        "Singapore": ["singapore", "changi"],
        "Bahrain": ["bahrain", "manama"],
        "Dubai": ["dubai", "jebel ali"],
        "Busan": ["busan"],
        "Guam": ["guam", "apra"],
        "Sasebo": ["sasebo", "juliet basin"],
        "Malaysia": ["malaysia", "klang"],
        "Philippines": ["philippines", "manila", "subic"],
        "Pascagoula": ["pascagoula"],
        "South China Sea": ["south china sea", "spratly islands", "luzon"],
        "Western Pacific (WESTPAC)": ["san bernardino strait", "western pacific", "westpac"],
        "Red Sea": ["red sea"],
        "Persian Gulf": ["persian gulf", "arabian gulf"],
        "Gulf of Oman": ["gulf of oman"],
        "Gulf of Aden": ["gulf of aden"],
        "Mediterranean": ["mediterranean"],
        "Caribbean Sea": ["caribbean", "st. croix", "trinidad", "tobago", "puerto rico"],
        "North Sea": ["north sea"],
        "Norwegian Sea": ["norwegian sea"],
        "Strait of Gibraltar": ["gibraltar"],
        "Suez Canal": ["suez"],
        "Bab el-Mandeb": ["bab el-mandeb"],
        "Philippine Sea": ["philippine sea", "okinawa"],
        "Atlantic Ocean": ["atlantic"],
        "Pacific Ocean": ["pacific"],
        "Indian Ocean": ["indian ocean"],
    }
    for label, keywords in location_map.items():
        if any(k in text for k in keywords): return label
    return "Underway / Unknown"

def extract_date(text: str) -> str:
    pattern = r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.?\s+\d{1,2}'
    matches = re.findall(pattern, text, re.IGNORECASE)
    return matches[-1] if matches else "Date Unspecified"

# ==============================================================================
# MODULE 2: SOCIAL INTEL LOGIC
# ==============================================================================

class IntelParser:
    @staticmethod
    def clean_text(text: str) -> str:
        text = re.sub(r'http\S+|www\.\S+', '', text)
        text = re.sub(r'\b\d{10,}\b', '', text)
        text = re.sub(r'(?i)posted|shared by|likes|comments|src:|twitter|instagram|facebook', '', text)
        text = re.sub(r'^\d{1,2}, 202', '202', text)
        return " ".join(text.split())

    @staticmethod
    def extract_ship_details(text: str) -> Tuple[Optional[str], Optional[str], str]:
        ship_pattern = r"(USS\s+[A-Z][a-z]+(?:\s+[A-Z]\.?\s?[a-z]*){0,3}(?:\s+\([A-Za-z]+[-\s]?\d+\))?)"
        match = re.search(ship_pattern, text)
        if match:
            full_ship_str = match.group(1)
            hull_match = re.search(r'\(([A-Za-z]+[-\s]?\d+)\)', full_ship_str)
            hull = hull_match.group(1).replace(" ", "") if hull_match else "Unknown"
            ship_name = re.sub(r'\s+\(.*\)', '', full_ship_str).strip()
            context = text.replace(full_ship_str, "").strip()
            context = re.sub(r'^[-:,.]+\s*', '', context)
            return ship_name, hull, context
        return None, None, text

def scrape_warshipcam() -> List[Dict]:
    print(f"    > Scanning WarshipCam.net...")
    intel_report = []
    try:
        response = requests.get(WARSHIPCAM_URL, headers={'User-Agent': USER_AGENT}, timeout=20)
        soup = BeautifulSoup(response.content, 'html.parser')

        raw_data = soup.get_text("\n", strip=True).split('\n')
        images = soup.find_all('img', alt=True)
        for img in images:
            if len(img['alt']) > 40: raw_data.append(img['alt'])

        seen_texts = set()
        date_pattern = re.compile(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.?\s+\d{1,2},?\s+202[4-6]', re.IGNORECASE)

        for text in raw_data:
            if "USS" not in text: continue
            cleaned = IntelParser.clean_text(text)
            if cleaned in seen_texts: continue

            date_match = date_pattern.search(text)
            if date_match:
                ship, hull, context = IntelParser.extract_ship_details(cleaned)
                if ship:
                    seen_texts.add(cleaned)
                    intel_report.append({
                        "Source": "WarshipCam.net",
                        "Date": date_match.group(0),
                        "Ship": ship,
                        "Hull": hull,
                        "Activity/Location": context[:150]
                    })
    except Exception as e:
        print(f"    ! Error scraping WarshipCam: {e}")
    return intel_report

def duckduckgo_snipe(query: str) -> List[Dict]:
    print(f"    > Sniping DuckDuckGo for: '{query}'...")
    search_url = "https://html.duckduckgo.com/html/"
    search_term = f"WarshipCam {query}"
    intel_report = []
    try:
        time.sleep(random.uniform(1.5, 3.0))
        response = requests.post(search_url, data={'q': search_term}, headers={'User-Agent': USER_AGENT}, timeout=15)
        soup = BeautifulSoup(response.content, 'html.parser')
        results = soup.find_all('div', class_='result__body')
        if not results: results = soup.find_all('a', class_='result__a')

        for res in results:
            text = res.get_text(" ", strip=True)
            if "No results found" in text: continue
            if "WarshipCam" in text and "USS" in text:
                cleaned = IntelParser.clean_text(text)
                ship, hull, context = IntelParser.extract_ship_details(cleaned)
                if ship:
                    date_match = re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.?\s+\d{1,2},?\s+202[4-6]', text)
                    date_str = date_match.group(0) if date_match else "Recent (Search Snippet)"
                    intel_report.append({
                        "Source": "DuckDuckGo/Search",
                        "Date": date_str,
                        "Ship": ship,
                        "Hull": hull,
                        "Activity/Location": context[:150]
                    })
    except Exception as e:
        print(f"    ! Error scraping DDG: {e}")
    return intel_report

# ==============================================================================
# MAIN EXECUTION
# ==============================================================================

def main():
    print(f"{'='*90}")
    print(f"UNIFIED NAVAL INTELLIGENCE COLLECTOR (Official + OSINT)")
    print(f"{'='*90}\n")

    # --- PART 1: OFFICIAL SCRAPER ---
    print(f"[*] PHASE 1: OFFICIAL FLEET TRACKER (USCarriers.net)")
    official_data = []
    for url in FLEET_URLS:
        hull_match = re.search(r'((?:cvn|lha|lhd)\d+)', url, re.IGNORECASE)
        hull = hull_match.group(1).upper() if hull_match else "UNK"

        raw_text = fetch_history_text(url)
        if "ERROR" in raw_text:
            year, status, loc_tag, date_str = "Error", raw_text, "Error", "Error"
        else:
            year, status = parse_status_entry(raw_text)
            loc_tag = categorize_location(status)
            date_str = extract_date(status)
            if date_str == "Date Unspecified": date_str = year

        official_data.append({"Hull": hull, "Location": loc_tag, "Date": date_str, "Status Sentence": status, "Source URL": url})
        print(f"    [{hull}] [{loc_tag}] [{date_str}] {status[:80]}...")

    # Write Official CSV
    with open("big_deck_status.csv", 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=["Hull", "Location", "Date", "Status Sentence", "Source URL"])
        writer.writeheader()
        writer.writerows(official_data)
    print(f"    >>> Official data saved to 'big_deck_status.csv'\n")

    # --- PART 2: SOCIAL INTEL ---
    print(f"[*] PHASE 2: SOCIAL INTELLIGENCE (OSINT)")
    web_intel = scrape_warshipcam()
    search_intel = duckduckgo_snipe('"USS" "2025"')
    all_intel = web_intel + search_intel

    # Deduping Social Data
    unique_ships = {}
    for item in all_intel:
        key = item['Ship']
        if key not in unique_ships or len(item['Activity/Location']) > len(unique_ships[key]['Activity/Location']):
            unique_ships[key] = item
    social_data = list(unique_ships.values())

    if social_data:
        print(f"    + Found {len(social_data)} unique social sightings.")
        with open("social_naval_intel.csv", 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=["Source", "Date", "Ship", "Hull", "Activity/Location"], extrasaction='ignore')
            writer.writeheader()
            writer.writerows(social_data)
        print(f"    >>> Social data saved to 'social_naval_intel.csv'")
    else:
        print("    ! No social sightings found.")

    print(f"\n{'='*90}")
    print(f"SUCCESS: Intelligence Cycle Complete.")
    print(f"Ready to map with 'master_fleet_mapper.py'")
    print(f"{'='*90}")

if __name__ == "__main__":
    main()

UNIFIED NAVAL INTELLIGENCE COLLECTOR (Official + OSINT)

[*] PHASE 1: OFFICIAL FLEET TRACKER (USCarriers.net)
    [CVN68] [South China Sea] [Nov. 18] From November 8-11, the Nimitz CSG conducted operations  off the coast of Brunei...
    [CVN69] [Norfolk / Portsmouth] [Jan. 8] September 26, USS Dwight D. Eisenhower moored at Pier 12N on Naval Station Norfo...
    [CVN70] [San Diego] [September 16] September 16, The Carl Vinson moored at Juliet Pier on Naval Air Station North I...
    [CVN71] [San Diego] [Nov. 10] November  8, USS Theodore Roosevelt moored at Berth Lima on Naval Air Station No...
    [CVN72] [Pacific Ocean] [2025] , USS Abraham Lincoln departed San Diego for a scheduled deployment....
    [CVN73] [South China Sea] [Nov. 22] From November 19-21, the George Washington CSG conducted operations northeast of...
    [CVN74] [Newport News] [April 8] April 8, 2024 USS John C. Stennis undocked and moored at Outfitting Berth #1 on ...
    [CVN75] [Norfolk / Portsmouth] [Oct. 8] S

In [11]:
"""
US NAVY MASTER FLEET MAPPER (v4.0 - Location Conflict Fix)

Integrates data from:
1. big_deck_status.csv (Official History)
2. social_naval_intel.csv (Social Media/WarshipCam)

Fixes v4.0:
- Manually offsets 'Norfolk' vs 'Newport News' coordinates to prevent
  visual stacking of pins in Hampton Roads.
- Ensures distinct visibility for all ports.
"""

import pandas as pd
import folium
import os
from folium.plugins import MarkerCluster, BeautifyIcon

# --- CONFIGURATION ---
OFFICIAL_DATA_FILE = "big_deck_status.csv"
SOCIAL_DATA_FILE = "social_naval_intel.csv"
OUTPUT_FILE = "master_fleet_map.html"

# --- HULL WHITELIST ---
TARGET_FLEET = [
    "CVN-68", "CVN-69", "CVN-70", "CVN-71", "CVN-72", "CVN-73",
    "CVN-74", "CVN-75", "CVN-76", "CVN-77", "CVN-78",
    "LHD-1", "LHD-2", "LHD-3", "LHD-4", "LHD-5", "LHD-7", "LHD-8",
    "LHA-6", "LHA-7"
]

# --- COORDINATE DICTIONARY ---
# Maps keywords to [Lat, Lon].
LOCATION_COORDS = {
    # US Ports / Shipyards
    # Offset Norfolk and Newport News slightly to prevent overlap
    "norfolk": [36.96, -76.30],        # Naval Station (East)
    "portsmouth": [36.82, -76.30],
    "newport news": [36.98, -76.46],   # Shipyard (West)
    "virginia beach": [36.85, -75.97], # Oceana Area

    "san diego": [32.68, -117.18],
    "north island": [32.70, -117.20],
    "camp pendleton": [33.29, -117.40],
    "bremerton": [47.55, -122.64],
    "kitsap": [47.70, -122.70],
    "everett": [47.98, -122.22],
    "mayport": [30.39, -81.42],
    "pearl harbor": [21.35, -157.97],
    "pascagoula": [30.34, -88.56],
    "ingalls": [30.34, -88.56],
    "groton": [41.39, -72.08],

    # Foreign Ports
    "yokosuka": [35.29, 139.66],
    "sasebo": [33.16, 129.71],
    "busan": [35.10, 129.11],
    "guam": [13.44, 144.65],
    "singapore": [1.30, 103.85],
    "changi": [1.35, 104.00],
    "bahrain": [26.22, 50.61],
    "dubai": [25.26, 55.30],
    "jebel ali": [25.00, 55.06],
    "manila": [14.59, 120.97],
    "subic": [14.80, 120.27],
    "okinawa": [26.30, 127.80],
    "klang": [3.00, 101.38],
    "trinidad": [10.66, -61.51],
    "tobago": [11.25, -60.67],
    "st. croix": [17.71, -64.88],
    "rostock": [54.15, 12.10],
    "plymouth": [50.37, -4.14],
    "oslo": [59.91, 10.75],
    "split": [43.50, 16.44],

    # Regions / Seas
    "south china sea": [12.00, 114.00],
    "spratly": [10.00, 114.00],
    "luzon": [16.00, 121.00],
    "philippine sea": [20.00, 130.00],
    "western pacific": [15.00, 135.00],
    "westpac": [15.00, 135.00],
    "san bernardino": [12.53, 124.28],
    "persian gulf": [27.00, 51.00],
    "arabian gulf": [27.00, 51.00],
    "red sea": [20.00, 38.00],
    "gulf of oman": [24.00, 58.00],
    "gulf of aden": [12.00, 48.00],
    "mediterranean": [35.00, 18.00],
    "adriatic": [42.00, 15.00],
    "caribbean": [15.00, -75.00],
    "puerto rico": [18.22, -66.59],
    "north sea": [56.00, 3.00],
    "norwegian sea": [66.00, 5.00],
    "gibraltar": [35.95, -5.60],
    "suez": [30.60, 32.33],
    "panama canal": [9.10, -79.67],

    # Oceans (Fallback)
    "atlantic": [33.00, -60.00],
    "pacific": [25.00, -150.00],
    "indian ocean": [-5.00, 80.00],
}

def get_location_key_and_coords(text):
    """
    Scans a raw text string against the coordinate dictionary.
    Returns: (location_key, [lat, lon]) or (None, None)
    """
    if not isinstance(text, str): return None, None
    text = text.lower()

    # Sort keys by length (descending) to ensure "Newport News" matches before "Newport"
    # and avoids partial matches.
    sorted_keys = sorted(LOCATION_COORDS.keys(), key=len, reverse=True)

    for location in sorted_keys:
        if location in text:
            return location, LOCATION_COORDS[location]
    return None, None

def is_target_ship(ship_name, hull):
    """Checks if a ship from the social feed is in our Big Deck whitelist."""
    s_hull = str(hull).upper().replace(" ", "")
    for target in TARGET_FLEET:
        if target.replace("-", "") in s_hull.replace("-", ""):
            return True
    return False

def generate_map():
    print(f"{'='*60}")
    print(f"MASTER FLEET MAPPER (v4.0 - Fixes Applied)")
    print(f"{'='*60}")

    locations_db = {}

    def add_to_db(loc_key, coords, ship_data):
        if loc_key not in locations_db:
            locations_db[loc_key] = {"coords": coords, "ships": []}
        locations_db[loc_key]["ships"].append(ship_data)

    # --- PROCESS OFFICIAL DATA ---
    if os.path.exists(OFFICIAL_DATA_FILE):
        print(f"Processing Official Data...")
        df_official = pd.read_csv(OFFICIAL_DATA_FILE)

        for _, row in df_official.iterrows():
            loc_tag = str(row['Location'])
            loc_key, coords = get_location_key_and_coords(loc_tag)

            if coords:
                add_to_db(loc_key, coords, {
                    "hull": row['Hull'],
                    "name": "",
                    "date": row['Date'],
                    "status": row['Status Sentence'],
                    "source": "Official",
                    "type": "CVN" if "CVN" in row['Hull'] else "LHD"
                })
            else:
                print(f"  ! Unmapped Official Ship: {row['Hull']} -> '{loc_tag}'")

    # --- PROCESS SOCIAL DATA ---
    if os.path.exists(SOCIAL_DATA_FILE):
        print(f"Processing Social Intel...")
        try:
            df_social = pd.read_csv(SOCIAL_DATA_FILE)
            for _, row in df_social.iterrows():
                ship_name = row.get('Ship', '')
                hull = row.get('Hull', '')
                raw_text = str(row.get('Activity/Location', '')) + " " + str(row.get('Raw Intel', ''))
                date_str = row.get('Date', 'Unknown')

                if is_target_ship(ship_name, hull):
                    loc_key, coords = get_location_key_and_coords(raw_text)
                    if coords:
                        add_to_db(loc_key, coords, {
                            "hull": hull,
                            "name": ship_name,
                            "date": date_str,
                            "status": raw_text[:150] + "...",
                            "source": "OSINT",
                            "type": "CVN" if "CVN" in str(hull).upper() else "LHD"
                        })
        except Exception as e:
            print(f"  ! Error reading social CSV: {e}")

    # --- RENDER MAP ---
    m = folium.Map(location=[25, 0], zoom_start=3, tiles="CartoDB positron")

    marker_count = 0

    for loc_key, data in locations_db.items():
        coords = data['coords']
        ships = data['ships']

        has_cvn = any(s['type'] == "CVN" for s in ships)
        if has_cvn:
            color = "blue"
            icon = "plane"
        else:
            color = "red"
            icon = "anchor"

        hull_list = [s['hull'] for s in ships]
        tooltip_text = f"{loc_key.title()}: {', '.join(hull_list)}"

        popup_html = f"""
        <div style="font-family:sans-serif; width:350px; overflow-y:auto; max-height:300px;">
            <h4 style="margin-bottom:5px; border-bottom:1px solid #ccc;">{loc_key.upper()}</h4>
            <table style="width:100%; font-size:11px; border-collapse:collapse;">
        """

        for s in ships:
            row_color = "#f9f9f9" if s['source'] == "Official" else "#eaffea"
            source_badge = "OFFICIAL" if s['source'] == "Official" else "SOCIAL"
            badge_color = "gray" if s['source'] == "Official" else "green"

            popup_html += f"""
            <tr style="background-color:{row_color}; border-bottom:1px solid #eee;">
                <td style="padding:5px; vertical-align:top;">
                    <b>{s['hull']}</b><br>
                    <span style="color:white; background:{badge_color}; padding:1px 3px; border-radius:3px; font-size:9px;">{source_badge}</span>
                </td>
                <td style="padding:5px;">
                    <b>{s['date']}</b><br>
                    {s['status']}
                </td>
            </tr>
            """

        popup_html += "</table></div>"

        folium.Marker(
            location=coords,
            popup=folium.Popup(popup_html, max_width=370),
            tooltip=tooltip_text,
            icon=folium.Icon(color=color, icon=icon, prefix='fa')
        ).add_to(m)
        marker_count += 1

    # Legend
    title_html = '''
             <div style="position: fixed;
                         top: 10px; right: 10px; width: 180px; height: 110px;
                         z-index:9999; font-size:13px;
                         background-color: white; opacity: 0.9;
                         padding: 10px; border-radius: 5px; border: 1px solid grey; box-shadow: 2px 2px 5px rgba(0,0,0,0.3);">
                 <b>US Navy Fleet Overview</b><br>
                 <hr style="margin:5px 0;">
                 <i class="fa fa-plane" style="color:blue; margin-right:5px;"></i> Carrier Present<br>
                 <i class="fa fa-anchor" style="color:red; margin-right:5px;"></i> Amphib Only<br>
                 <br>
                 <span style="font-size:10px;">*Click pins for ship list</span>
             </div>
             '''
    m.get_root().html.add_child(folium.Element(title_html))

    m.save(OUTPUT_FILE)
    print(f"\n{'='*60}")
    print(f"SUCCESS: Aggregated Map generated at '{os.path.abspath(OUTPUT_FILE)}'")
    print(f"Mapped {len(locations_db)} unique fleet locations.")
    print(f"{'='*60}")

if __name__ == "__main__":
    generate_map()

MASTER FLEET MAPPER (v4.0 - Fixes Applied)
Processing Official Data...
Processing Social Intel...

SUCCESS: Aggregated Map generated at '/content/master_fleet_map.html'
Mapped 8 unique fleet locations.
